In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-03-20T12:33:53.126606-04:00

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 8.10.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 22.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import random
import sklearn
import gc
import os

In [4]:
%watermark --iversions

numpy  : 1.23.5
pandas : 1.5.3
sklearn: 1.2.1



In [5]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [6]:
train.head()

label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      5    0    0    0    0    0    0    0    0    0  ...      0      0   
1      0    0    0    0    0    0    0    0    0    0  ...      0      0   
2      4    0    0    0    0    0    0    0    0    0  ...      0      0   
3      1    0    0    0    0    0    0    0    0    0  ...      0      0   
4      9    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[5 rows x 785 columns]

In [7]:
test.head()

label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      7    0    0    0    0    0    0    0    0    0  ...      0      0   
1      2    0    0    0    0    0    0    0    0    0  ...      0      0   
2      1    0    0    0    0    0    0    0    0    0  ...      0      0   
3      0    0    0    0    0    0    0    0    0    0  ...      0      0   
4      4    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[5 rows x 785 columns]

In [18]:
columns = list(train.columns[1:])

In [22]:
random.shuffle(columns)

In [25]:
train[columns].head()

9x15  21x12  23x18  20x8  18x21  6x26  18x4  7x1  26x17  4x4  ...  24x2  \
0   198    114      0     0     64     0     0    0      0    0  ...     0   
1   252     29      0   253    225     0     0    0      0    0  ...     0   
2     0      0    169     0      0     0     0    0      0    0  ...     0   
3     0    253      0     0      0     0     0    0      0    0  ...     0   
4   253      0      0     0      0     0     0    0    252    0  ...     0   

   17x25  7x9  12x10  6x2  4x28  28x24  21x8  28x7  26x4  
0      0   30      0    0     0      0     0     0     0  
1      0    0    165    0     0      0   252     0     0  
2      0    0      0    0     0      0     0     0     0  
3      0    0      0    0     0      0     0     0     0  
4      0    0    253    0     0      0     0     0     0  

[5 rows x 784 columns]

In [26]:
test[columns].head()

9x15  21x12  23x18  20x8  18x21  6x26  18x4  7x1  26x17  4x4  ...  24x2  \
0   198      0      0     0      0     0     0    0      0    0  ...     0   
1   209    253    155     0      0     0     0    0      0    0  ...     0   
2     0     61      0     0      0     0     0    0      0    0  ...     0   
3   251    251      0   218    221     0     0    0      0    0  ...     0   
4     0      0    252     0      0     0     0    0      0    0  ...     0   

   17x25  7x9  12x10  6x2  4x28  28x24  21x8  28x7  26x4  
0      0    0      0    0     0      0     0     0     0  
1      0  250      0    0     0      0     0     0     0  
2      0    0      0    0     0      0     0     0     0  
3      0    0    251    0     0      0   113     0     0  
4      0    0    211    0     0      0     0     0     0  

[5 rows x 784 columns]

In [27]:
X = train[columns].values
X_test = test[columns].values
Y = train.label.values

In [28]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [29]:
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = MLPClassifier()
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold accuracy: 0.9675833333333334
Fitting fold 2
Fold accuracy: 0.96225
Fitting fold 3
Fold accuracy: 0.9561666666666667
Fitting fold 4
Fold accuracy: 0.9598333333333333
Fitting fold 5
Fold accuracy: 0.961


In [30]:
print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

0.9613666666666667


In [12]:
0.9775833333333334

0.9775833333333334

In [31]:
preds = np.argmax(test_preds, axis=1)
preds

array([7, 2, 1, ..., 4, 5, 6])

In [32]:
test.label.values

array([7, 2, 1, ..., 4, 5, 6])

In [33]:
accuracy_score(test.label.values, preds)

0.9804

In [34]:
X.max()

255

In [35]:
X_test.max()

255

In [36]:
X = X/255.
X_test = X_test/255.
X.max()

1.0

In [37]:
X_test.max()

1.0

In [38]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = MLPClassifier()
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

Y_test = test.label.values

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.9780833333333333
Fitting fold 2
Fold accuracy: 0.9739166666666667
Fitting fold 3
Fold accuracy: 0.974
Fitting fold 4
Fold accuracy: 0.9765833333333334
Fitting fold 5
Fold accuracy: 0.9738333333333333
0.9752833333333333
0.9819
CPU times: user 1h 10min 16s, sys: 6min 43s, total: 1h 16min 59s
Wall time: 3min 57s
